#Pine cone and rag and vector basics
in pinecone is a vector database and what is vector , vector is a numeric way of expressing data like how to express the data for the computer to understand and map it accordingly like for a car it gives random number of 40 and red car 40.1 and blue 40.2 so this is how it works, so pinecone is the vector databases of these numbers and how they work ,

In [2]:
%pip install -qU langchain-pinecone pinecone-notebooks

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 16.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.8/244.8 kB 18.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.4/85.4 kB 7.0 MB/s eta 0:00:00


In [3]:
%pip install -qU langchain-pinecone langchain-google-genai


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.3/41.3 kB 1.5 MB/s eta 0:00:00


In [17]:
from google.colab import userdata
import os
from pinecone import Pinecone, ServerlessSpec

# Set your Pinecone API key here. Either paste it directly or retrieve it from userdata or a secret manager.
os.environ["PINECONE_API_KEY"] = userdata.get("PINECONE_API_KEY")
pinecone_api_key = os.environ.get("PINECONE_API_KEY")

# Initialize the Pinecone client
pc = Pinecone(api_key=pinecone_api_key)

In [16]:
import time

index_name = "my-rag-project"  # change if desired

pc.create_index(
  name=index_name,
  dimension=768,
  metric="cosine",
  spec=ServerlessSpec(cloud="aws", region="us-east-1"),
)

index = pc.Index(index_name)

In [18]:
from langchain_google_genai import GoogleGenerativeAIEmbeddings
import os

os.environ["GOOGLE_API_KEY"] = userdata.get("GOOGLE_API_KEY")
embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001")


In [19]:
vector = embeddings.embed_query("This is a test query tasmeer.")


In [20]:
vector[:5]


[0.02038930542767048,
 -0.06138036400079727,
 -0.03236711025238037,
 -0.04639166221022606,
 0.0323028527200222]

In [21]:
from langchain_pinecone import PineconeVectorStore

vector_store = PineconeVectorStore(index=index, embedding=embeddings)

In [22]:
from langchain_core.documents import Document

document_1 = Document(
    page_content="I had chocalate chip pancakes and scrambled eggs for breakfast this morning.",
    metadata={"source": "tweet"},
)

In [23]:
from uuid import uuid4

from langchain_core.documents import Document

document_1 = Document(
    page_content="I had chocalate chip pancakes and scrambled eggs for breakfast this morning.",
    metadata={"source": "tweet"},
)

document_2 = Document(
    page_content="The weather forecast for tomorrow is cloudy and overcast, with a high of 62 degrees.",
    metadata={"source": "news"},
)

document_3 = Document(
    page_content="Building an exciting new project with LangChain - come check it out!",
    metadata={"source": "tweet"},
)

document_4 = Document(
    page_content="Robbers broke into the city bank and stole $1 million in cash.",
    metadata={"source": "news"},
)

document_5 = Document(
    page_content="Wow! That was an amazing movie. I can't wait to see it again.",
    metadata={"source": "tweet"},
)

document_6 = Document(
    page_content="Is the new iPhone worth the price? Read this review to find out.",
    metadata={"source": "website"},
)

document_7 = Document(
    page_content="The top 10 soccer players in the world right now.",
    metadata={"source": "website"},
)

document_8 = Document(
    page_content="LangGraph is the best framework for building stateful, agentic applications!",
    metadata={"source": "tweet"},
)

document_9 = Document(
    page_content="The stock market is down 500 points today due to fears of a recession.",
    metadata={"source": "news"},
)

document_10 = Document(
    page_content="I have a bad feeling I am going to get deleted :(",
    metadata={"source": "tweet"},
)

documents = [
    document_1,
    document_2,
    document_3,
    document_4,
    document_5,
    document_6,
    document_7,
    document_8,
    document_9,
    document_10,
]


In [25]:
from uuid import uuid4

from langchain_core.documents import Document

In [24]:
uuids = [str(uuid4()) for _ in range(len(documents))]

vector_store.add_documents(documents=documents, ids=uuids)

['8fa42b78-035b-4419-bfc4-e01381817253',
 '4c6bda04-3c7c-46a9-a127-641f449d6cb8',
 '7a15ea93-880b-4220-872c-9d0dd37b7b35',
 'bde70f10-b485-4dd2-84c6-4de966ed4fe5',
 'a7b43911-6d90-49ca-82e5-2ca55b20013a',
 'f7bd1908-0bd6-4409-b315-3c6a89f1a206',
 '962fce3d-3e77-4223-877f-f897f6b600a8',
 '65f4b507-0ce6-49fa-831a-23ff1dbb7572',
 '87a8b7e0-75cf-49b8-b005-3607694a2ff5',
 'f799aff1-9768-4e32-ab35-bdd50ecc927b']

In [26]:
results = vector_store.similarity_search(
    "LangChain provides abstractions to make working with LLMs easy",
)
for res in results:
    print(f"* {res.page_content} [{res.metadata}]")

* LangGraph is the best framework for building stateful, agentic applications! [{'source': 'tweet'}]
* Building an exciting new project with LangChain - come check it out! [{'source': 'tweet'}]
* I have a bad feeling I am going to get deleted :( [{'source': 'tweet'}]
* I had chocalate chip pancakes and scrambled eggs for breakfast this morning. [{'source': 'tweet'}]


In [31]:
results = vector_store.similarity_search_with_score(
    "is cristiano roanldo the goat",
)
for res, score in results:
    print(f"* [SIM={score:3f}] {res.page_content} [{res.metadata}]")

* [SIM=0.566137] The top 10 soccer players in the world right now. [{'source': 'website'}]
* [SIM=0.528850] I have a bad feeling I am going to get deleted :( [{'source': 'tweet'}]
* [SIM=0.527474] Is the new iPhone worth the price? Read this review to find out. [{'source': 'website'}]
* [SIM=0.507011] Wow! That was an amazing movie. I can't wait to see it again. [{'source': 'tweet'}]


In [32]:
def answer_to_user(query: str):
  results = vector_store.similarity_search(query, k=2)
  final_answer= model(results,query)
  return final_answer